In [3]:
#计算图是现代深度学习框架的核心
#手动求取反向传播废时费力，而且容易出错
#autograd.Variable封装了data,grad,grad_fn三个数据
#data是tensor，grad的形状与data一致，grad_fn指向一个Func记录操作历史

#可选参数
#requires_grad(bool):是否对该ariable求导数
#volatile(bool):设置为True，构建在该variable上的图都不会求道，为推理阶段设计


In [26]:
import torch as t
from torch.autograd import Variable as var
from torch.autograd import Function

#从tensor中创建var，指定需要求导
a = var(t.ones(3,4),requires_grad = True)
b = var(t.zeros(3,4))
print(a)
print(b)

tensor([[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]], requires_grad=True)
tensor([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]])


In [5]:
c = a+b
print(c)
c = a.add(b)
print(c)
#上面两种方法一样结果

tensor([[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]], grad_fn=<AddBackward0>)
tensor([[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]], grad_fn=<AddBackward0>)


In [6]:
d = c.sum()
d.backward()#反向传播

In [7]:
#注意区别，第一种因为先取了data部分再sum，所以数据类型是tensor
c.data.sum()
#第二种因为直接sum,所以数据类型仍然为var
c.sum()

tensor(12., grad_fn=<SumBackward0>)

In [8]:
#例子求y=x^2 * e^x的导数
x = var(t.randn(3,4),requires_grad=True)

def f(x):
    '''定义y'''
    y = x**2 * t.exp(x)
    return y

y = f(x)

def gradf(x):
    '''手动求导'''
    dx = 2*x*t.exp(x) +x**2*t.exp(x)
    return dx

In [9]:
print("手动求导数:")
dx = gradf(x)
print(dx)
y.backward(t.ones(y.size()))#确保grad_var的形状与y一样
print("自动求导数：")  
x.grad

手动求导数:
tensor([[-0.1885, -0.1643,  2.3835, 12.7910],
        [20.3103, -0.4363, -0.3132, -0.2113],
        [ 1.2361, -0.2959, -0.3196,  6.1649]], grad_fn=<AddBackward0>)
自动求导数：


tensor([[-0.1885, -0.1643,  2.3835, 12.7910],
        [20.3103, -0.4363, -0.3132, -0.2113],
        [ 1.2361, -0.2959, -0.3196,  6.1649]])

In [14]:
x = var(t.ones(1))
b = var(t.rand(1),requires_grad = True)
w = var(t.rand(1),requires_grad = True)
y = w*x
z = y+b
#只要涉及到需要求道的就会自动变为求导
x.requires_grad,b.requires_grad,w.requires_grad,z.requires_grad

(False, True, True, True)

In [18]:
#grad_fn可以查看这个var的反向传播函数，比如z的反向传播函数就是add backward
#而y是乘法反向
z.grad_fn,y.grad_fn

(<AddBackward0 at 0x7f5638c2d310>, <MulBackward0 at 0x7f56395691d0>)

In [22]:
#像叶子节点，无法反向的，就会成为none
print(x.grad_fn)

None
